In [ ]:

%pylab inline

import tensorflow as tf
%matplotlib inline
import numpy as np
from collections import deque
from os import listdir
import functools

pylab.rcParams['figure.figsize'] = (4.0, 4.0)

import pylab
from tsne import bh_sne
from matplotlib import pyplot as plt
import pandas
import scipy
import numpy as np
import sklearn.manifold
import os
import gc
from time import gmtime, strftime
import seaborn as sns
from os import listdir
from os.path import isfile, join
import math
from scipy.stats.stats import pearsonr
import random as rand
import pyproj
import functools
import pickle 

image_width = 128

def display(image, min = 0.0, max = 1.0):
    plt.imshow(image, cmap = plt.get_cmap('gray'), interpolation='nearest')
    plt.show()
    




def display(image):
    plt.imshow(image, cmap = plt.get_cmap('gray'), interpolation='nearest')
    plt.show()
    
def memoize(obj, maxlen = 2000):
    """A decorator to cache advice objects using the advice key"""
    cache = obj.cache = {}
    deck = obj.deck = deque([], maxlen = maxlen)

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = args[0]
        if key not in cache:
            if len(deck) == deck.maxlen:
              del cache[deck.popleft()[0]]
            temp = obj(*args, **kwargs)
            cache[key] = temp
            deck.append((key, temp))
        return cache[key]
        
    return memoizer

@memoize
def load_mat_from_file(key):
    f = file("/home/isaac/Desktop/devika/ARCGIS/ArcGis/pickle_files/"+key+".npy","rb")
    return np.load(f)

@memoize
def get_file_containing(housex, housey):
#         print "house coords:", housex, housey
    for key in file_attributes:
        block = file_attributes[key]
    #         print block['xllcorner'], block['yllcorner']
        if block['xllcorner'] < housex < block['xllcorner'] + block['width']:
            if block['yllcorner'] < housey < block['yllcorner'] + block['height']:
                return key
    assert(1==0)

def imagex(housex, housey, block):
    return (int(housex)-block['xllcorner'])/5
def imagey(housex, housey, block):
    return (int(housey)-block['yllcorner'])/5



#housex and housey in meters
def get_heightmap_around(housex, housey, window_width_pixels = 28):
    
    def image_slice(key, housex, housey, radius = window_width_pixels/2):
        x = housex
        y = housey
        block = file_attributes[key]
        mat = load_mat_from_file(key)
        x1 = np.max([0,              imagex(x,y,block) - radius])
        x2 = np.min([block['ncols'], imagex(x,y,block) + radius])
        
        y1 = np.max([0,              imagey(x,y,block) - radius])
        y2 = np.min([block['nrows'], imagey(x,y,block) + radius])
        
        assert(x1 <= x2)
        assert(y1 <= y2)
        return  mat[block['nrows'] - y2:block['nrows'] - y1, x1:x2]
    
    window_width_feet = window_width_pixels * 5
    housex /= .3048 #convert to feet
    housey /= .3048
    x = housex
    y = housey
    ul = get_file_containing(x - window_width_feet/2, y - window_width_feet/2)
    ur = get_file_containing(x + window_width_feet/2, y - window_width_feet/2)
    ll = get_file_containing(x - window_width_feet/2, y + window_width_feet/2)
    lr = get_file_containing(x + window_width_feet/2, y + window_width_feet/2)

    for key in [ul, ur, ll, lr]:
        image = image_slice(key, x, y)
#         print image.shape
        if image.shape == (window_width_pixels, window_width_pixels):
            return image

    

    if ul == ur and lr == ll : # horizontal split


        result = np.zeros([window_width_pixels, window_width_pixels])
        upper_slice = image_slice(ul, housex, housey)
        lower_slice = image_slice(ll, housex, housey)
        
        result[:upper_slice.shape[0], :] = upper_slice
        result[ window_width_pixels - lower_slice.shape[0]:, :] = lower_slice


        return result
    elif ul == ll and ur == lr and ul != ur and ll != lr: # vertical split
        result = np.zeros([window_width_pixels, window_width_pixels])
        left_slice = image_slice(ll, housex, housey)
        right_slice = image_slice(ur, housex, housey)
#         print left_slice.shape
#         print right_slice.shape
        
        result[:, :left_slice.shape[1]] = left_slice
        result[:, window_width_pixels - right_slice.shape[1]:] = right_slice

#         plt.imshow(result)
#         plt.show()
#         display(result)

        return result
    else: # four way split
#         print "four way"
        ll_slice = image_slice(ll, housex, housey)
        ul_slice = image_slice(ul, housex, housey)
        lr_slice = image_slice(lr, housex, housey)
        ur_slice = image_slice(ur, housex, housey)
        
        result = np.zeros([window_width_pixels, window_width_pixels])
        result[:ll_slice.shape[0], :ll_slice.shape[1]] = ll_slice
        result[:lr_slice.shape[0], window_width_pixels - lr_slice.shape[1]:] = lr_slice
        
        result[window_width_pixels - ul_slice.shape[0]:, :ul_slice.shape[1]] = ul_slice
        result[window_width_pixels - ur_slice.shape[0]:, window_width_pixels - ur_slice.shape[1]:] = ur_slice
#         display(result)
        return result

        

def window_violated_chunk_borders(housex, housey):
    corners = [(housex-window_width_feet/2, housey-window_width_feet/2),
              (housex-window_width_feet/2, housey+window_width_feet/2),
              (housex+window_width_feet/2, housey+window_width_feet/2),
              (housex+window_width_feet/2, housey-window_width_feet/2)]
    files = map(lambda x: get_file_containing(x[0], x[1]), corners)
    return len(set(files)) > 1



file_attributes = {}

mypath = "/home/isaac/Desktop/devika/ARCGIS/ArcGis/ascii_files"
for filename in [f for f in listdir(mypath) if isfile(join(mypath, f))]: 
    attributes = {}
    with open(mypath + "/" + filename) as FileObj:
        for index, line in enumerate(FileObj):
            if(index < 6):
#                 print line
                attributes[line.split(" ")[0]] = int(line.split(" ")[-1][:-2])
            else:
                break # don't load the other lines into memory becuase that's a waste of time.

    attributes['width'] = attributes['ncols'] * attributes['cellsize']
    attributes['height'] = attributes['nrows'] * attributes['cellsize']
    file_attributes[filename] = attributes
print "loaded all file header attributes into dict"


def load_data(path, normalize_columns = False, only_columns_containing = ""):
    print("loading...")
    gc.collect() # collect garbage
    data = pandas.read_hdf(path, '/df')
    df = pandas.DataFrame(data)
    df = df.sort(['hcad']).fillna(0)
    df = df.reset_index()#(ascending=True)

    data_dict = {}
    for label in set(df._get_numeric_data().columns).union({'hcad'}):
        # union hcad to ensure that hcad col comes in even if not considered numerical
        if normalize_columns and  label != 'hcad':
            column_data = np.array(df[label].astype(float))
            column_data -= np.min(column_data)
            data_dict[label] = column_data / np.max(column_data)
        else:
            data_dict[label] = df[label].astype(float)

        result = pandas.DataFrame.from_dict(data_dict)

    result = result.replace([np.inf, -np.inf], 1)
    
    
    for label in result:
        if not only_columns_containing in str(label):
            result.drop([str(label)], axis = 1 , inplace= True)
    return result




def get_batch(batch_size = 100, random = True, start_index = 0, val = False):
    metamat = np.zeros([batch_size, image_width, image_width, 2]).astype(float32)
    regression_y = np.zeros([batch_size,1]).astype(float32)
    categorical_y = np.zeros([batch_size]).astype(int32)
    extra_features = np.zeros([batch_size,len(wind_data[0])]).astype(float32)
    i = 0
    for _ in range(batch_size):

        index = i + start_index
        if random:
            index = rand.randint(0, len(meta)-1)
            if index % 100 == 0 and not val:
                index +=  rand.randint(1, 99)
                if index > len(meta):
                    index = len(meta) -1
            elif val:
                index -= index % 100
        lon = [meta['lon'][index]]
        lat = [meta['lat'][index]]
        max_wind_speed = np.max(wind_data[index])
        regression_y[i] = y_gaussian[index]
        categorical_y[i] = y_data['y'][index]
        extra_features[i] = wind_data[index]

        xx, yy = pyproj.transform(wgs84, UTM26N, lon, lat)
        housex=xx[0]
        housey=yy[0]

        lidar = None
        try:
            lidar = get_heightmap_around(housex, housey, window_width_pixels=image_width)
        except:
            continue
        if(lidar == None):
            continue
        
        lidar = lidar.copy()
        lidar[lidar < -100] = -100
        lidar[lidar >250] = 250
        lidar /= 250.0
#         lidar -= np.min(lidar)
#         lidar /= np.max(lidar)


        metamat[i, :, :, 0] = lidar

        wind_data_channel = np.zeros([image_width, image_width]).astype(float32)
        wind_data_channel.fill(max_wind_speed)
        metamat[i, :, :, 1] = wind_data_channel



        i += 1

    #metamat = metamat.reshape([batch_size, image_width**2])

    return metamat, extra_features, regression_y, np.eye(2)[categorical_y]


# metamat, extra_features, regression_y, cat_y = get_batch(1, random = False, start_index=12314)

# meta = load_data("/home/isaac/Dropbox/data_for_brian/meta/df_meta.hd")

# X, _, _ = get_batch(100)
# plt.imshow(X[0,:,:,0])
# plt.show()
# X = X[:,:,:,0].reshape([-1])
# print np.min(X)
# plt.hist(X, bins = 50)
# plt.show()

In [ ]:



wind_data = load_data("/home/isaac/Dropbox/data_for_brian/wind_features/hcad_interp_withoutpartial_rad100_hist8x8.mat.hd", 
              normalize_columns=True, only_columns_containing = "spd")
meta = load_data("/home/isaac/Dropbox/data_for_brian/meta/df_meta.hd")
y_data = load_data("/home/isaac/Dropbox/data_for_brian/y_df.hd")
hcad_dpr = load_data("/home/isaac/Dropbox/data_for_brian/hcad_features/hcad_df_200.hd", 
              normalize_columns=True, only_columns_containing = "mean_dpr")
wind_data = np.concatenate((wind_data, hcad_dpr), axis = 1)
# wind_data = wind_data[meta['img0'] == 1]
# y_data = wind_data[meta['img0'] == 1]
y_gaussian = pickle.load(open( "/home/isaac/Desktop/house_guassian_damage_balance.p", "rb" ) )

wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
UTM26N=pyproj.Proj("+init=EPSG:2278") # UTM coords, zone Texas Central, WGS84 datum

# lon = [meta['lon'][250000]] #these are good becuase of clear local features
# lat = [meta['lat'][250000]]


lon = [meta['lon'][12000]]
lat = [meta['lat'][12000]]

print "house lat lon", lat, lon
xx, yy = pyproj.transform(wgs84, UTM26N, lon, lat)
print xx, yy
housex=xx[0]
housey=yy[0]
display(get_heightmap_around(housex,housey, window_width_pixels = image_width))

n=10000



In [ ]:
import tflearn
tf.reset_default_graph()
# Data loading and preprocessing
# image_data_data, extra_features_data, batch_y = get_batch(batch_size=13640, random = True)
image_data_data, _, batch_y_regression, batch_y_categorical = get_batch(batch_size=160, random = True)
# print batch_y[0]
# np.random.shuffle(batch_y)
# print batch_y[0]
# batch_y = np.round(batch_y)
# for i in range(len(batch_y)):
#     extra_features_data[i].fill(batch_y[i][0])

image_data = tf.placeholder(tf.float32, [None, image_width, image_width, 2], name = 'image_data_placeholder')
extra_features = tf.placeholder(tf.float32, [None, len(wind_data[0])], name = 'extrafeatures_ph')
# extra_features = tf.placeholder(tf.float32, [None, 1], name = 'extrafeatures_ph')

regression_y_in = tf.placeholder(tf.float32, [None, 1], name = 'y_regression_ph')
categorical_y_in = tf.placeholder(tf.float32, [None, 2], name = 'y_categorical_ph')



print image_data.get_shape(), "<- image input"

network = tflearn.conv_2d(image_data, 8, 5, strides=2, activation='relu', regularizer="L2", weight_decay=0.0)
bottom_conv = network
network = tflearn.conv_2d(network, 8, 5, strides=2, activation='relu', regularizer="L2", weight_decay=0.0)

middle = network

network = tflearn.conv_2d_transpose(network, 8, 5, [image_width, image_width, 8], strides=4, activation='relu', 
                                      regularizer="L2", weight_decay=0.0)

AE_output = tflearn.conv_2d_transpose(network, 2, 5, [image_width, image_width, 2], strides=1, activation='sigmoid', 
                                      regularizer="L2", weight_decay=0.0)


# network = tflearn.conv_2d(image_data, 32, 11, strides=2, activation='relu', regularizer="L2", weight_decay=0.0)
# bottom_conv = network
# network = tflearn.conv_2d(network, 32, 11, strides=2, activation='relu', regularizer="L2", weight_decay=0.0)

# print network.get_shape()
# network = tflearn.layers.conv.max_pool_2d (network, 2, strides=2, padding='same', name='MaxPool2D')
# print network.get_shape()
# network = tflearn.conv_2d(network, 128, 5,  strides=1, activation='relu', regularizer="L2", weight_decay=0.0)
# network = tflearn.conv_2d(network, 128, 5,  strides=1, activation='relu', regularizer="L2", weight_decay=0.0)

# print network.get_shape()
# netowrk = tflearn.layers.core.dropout (network, .7, name='Dropout')

# network = tflearn.layers.conv.max_pool_2d (network, 2, strides=2, padding='same', name='MaxPool2D')
# print network.get_shape()

# network = tflearn.conv_2d(network, 128, 3,  strides=1, activation='relu', regularizer="L2", weight_decay=0.0)
# network = tflearn.conv_2d(network, 128, 3,  strides=1, activation='relu', regularizer="L2", weight_decay=0.0)

# print network.get_shape()
# netowrk = tflearn.layers.core.dropout (network, .7, name='Dropout')

# network = tflearn.conv_2d(network, 64, 3, activation='relu', regularizer="L2", weight_decay=0.0)

# middle = network

# print middle.get_shape(), "<- middle"


# ## Reconstruct the input
# network = tflearn.conv_2d_transpose(middle, 32, 5, [16, 16, 32], strides=2, activation='relu', regularizer="L2", weight_decay=0.0)
# print network.get_shape()
# network = tflearn.layers.conv.upsample_2d (network, 2)
# print network.get_shape()

# network = tflearn.conv_2d_transpose(network, 32, 5, [64, 64, 32], strides=2, activation='relu', regularizer="L2", weight_decay=0.0)
# print network.get_shape()
# print network.get_shape()
# AE_output = tflearn.conv_2d_transpose(network, 2, 5, [image_width, image_width, 2], strides=2, activation='tanh', regularizer="L2", weight_decay=0.0)
# print AE_output.get_shape()


## predict damage
dense1 = tflearn.fully_connected(middle, 460, activation='relu',
                                 regularizer='L2', weight_decay=0.0)
print "pre merge", dense1.get_shape()
dense2 = tflearn.layers.merge_ops.merge ([dense1, extra_features], 'concat', axis = 1, name='MergeOutputs')
print "post merge", dense2.get_shape()

dense3 = tflearn.fully_connected(dense2, 130, activation='relu',
                                 regularizer='L2', weight_decay=0.0)

d = tflearn.fully_connected(dense2, 130, activation='relu',
                                 regularizer='L2', weight_decay=0.0)

d = tflearn.fully_connected(d, 130, activation='relu',
                                 regularizer='L2', weight_decay=0.0)

y_regression_pred = tflearn.fully_connected(d, 1, activation='sigmoid',
                                 regularizer='L2', weight_decay=0.0)


# y_categorical_pred = tflearn.fully_connected(d, 2, activation='softmax',
#                                  regularizer='L2', weight_decay=0.0)


# Define loss and optimizer
y_regression_loss = tf.reduce_mean(tf.square(tf.sub(y_regression_pred, regression_y_in))) 
# cross_entropy = 0.143 * tf.reduce_mean(-tf.reduce_sum(categorical_y_in * tf.log(y_categorical_pred), reduction_indices=[1]))
average_conv_weights = tf.reduce_mean(tf.square(bottom_conv.W)) + tf.reduce_mean(tf.square(AE_output.W))
autoencoder_loss = tf.reduce_mean(tf.square(tf.sub(image_data, AE_output)) ) + .001 * average_conv_weights
autoencoder_optimizer = tf.train.RMSPropOptimizer(learning_rate=.0004).minimize(autoencoder_loss)

regression_alpha = .7
categorical_alpha = 0
cost = ((regression_alpha *y_regression_loss ) 
#     + (categorical_alpha * cross_entropy) 
    + ((1 - regression_alpha + categorical_alpha) * autoencoder_loss))

optimizer = tf.train.RMSPropOptimizer(learning_rate=.0004).minimize(cost)

# y_regression_optimizer = tf.train.RMSPropOptimizer(learning_rate=.02).minimize(cost)

# dense2 = tflearn.fully_connected(dense1, 164, activation='relu',
#                                  regularizer='L2', weight_decay=0.0)
# network = tflearn.fully_connected(network, 1, activation='tanh')

# # Regression using SGD with learning rate decay and Top-3 accuracy
saver = tf.train.Saver()

init = tf.initialize_all_variables()
# Launch the graph
sess = tf.Session()
sess.run(init)




# sgd = tflearn.RMSProp(learning_rate=0.001)
# net = tflearn.regression(network, optimizer=sgd,
#                          loss='mean_square')

# # Training
# model = tflearn.DNN(net, tensorboard_verbose=0)


In [ ]:
bottom_conv_weights = sess.run(bottom_conv.W)
print bottom_conv_weights.shape
for i in range(bottom_conv_weights.shape[-1]):
    plt.imshow(bottom_conv_weights[:,:,0,i])
    plt.show()

In [ ]:
batch_size = 200
step = 1
image_data_data, extra_features_data, batch_y_regression, batch_y_categorical \
    = get_batch(batch_size=400, random = False, start_index = 10000)


#print "batch_y:", batch_y
# plt.scatter(extra_features_data[0], extra_features_data[1])
# plt.show() 

def show_image(image_mat, index):
    print np.min(image_mat[index,:,:,0]),np.max(image_mat[index,:,:,0])

    plt.imshow(image_mat[index,:,:,0])
    plt.show()

    
# show_image(image_data_data, 0)
# show_image(image_data_data, 1)
# show_image(image_data_data, 2)
# show_image(image_data_data, 3)


# step = 0
print "image shape:", image_data_data.shape
while step * batch_size < 10000000000:
    image_data_data, extra_features_data, batch_y_regression, batch_y_categorical\
            = get_batch(batch_size=200, random = True, start_index = 0)  


    _, batch_cost = sess.run([autoencoder_optimizer, autoencoder_loss], feed_dict={image_data: image_data_data,
                                   extra_features: extra_features_data,
                                  regression_y_in: batch_y_regression,
                                   categorical_y_in: batch_y_categorical })
    batch_costs.append(batch_cost)
    if  step % 500 == 0:
        saver.save(sess, 'joint_conv_model', global_step=step)
        image_data_data, extra_features_data, batch_y_regression, batch_y_categorical\
            = get_batch(batch_size=100, random = True, start_index = 0)      
            
        print "mean MSE:", np.mean(np.square(np.array([y_average]*len(batch_y_regression)) - batch_y_regression))

        # Calculate batch loss and accuracy
        y_guess, y_loss_fetched, autoencoder_loss_fetched, joint_loss, autoencoder_result = \
                                            sess.run([y_regression_pred, y_regression_loss, autoencoder_loss, cost, AE_output], 
                                              feed_dict={image_data: image_data_data,
                                                          extra_features: extra_features_data,
                                                          regression_y_in: batch_y_regression,
                                                           categorical_y_in: batch_y_categorical })
        print ("Iter " + str(step*batch_size) + ", Minibatch Y Loss= ", 
            str(y_loss_fetched), "Joint cost:", str(joint_loss),
               "Autoencoder cost:", str(autoencoder_loss_fetched))
        
        print np.max(autoencoder_result), np.min(autoencoder_result)

        plt.imshow(autoencoder_result[0,:,:,0])
        plt.show()
        plt.imshow(image_data_data[0,:,:,0])
        plt.show()
        plt.scatter(list(batch_y_regression.reshape([-1])), y_guess)
        plt.show()
        moving_window_size = 5
        #plt.plot(batch_costs[moving_window_size:-moving_window_size])
        plt.plot(np.convolve(batch_costs, 
                             np.ones(moving_window_size)/moving_window_size)[moving_window_size:-moving_window_size])
        plt.show()
        
        
        image_data_data, extra_features_data, batch_y_regression, batch_y_categorical\
            = get_batch(batch_size=200, random = True, start_index = 0, val = True)  
        _, y_val_loss_fetched, _, _ = \
                    sess.run([y_regression_pred, y_regression_loss, autoencoder_loss, cost], 
                              feed_dict={image_data: image_data_data,
                                          extra_features: extra_features_data,
                                          regression_y_in: batch_y_regression,
                                           categorical_y_in: batch_y_categorical })
        print "mean MSE on val data:", np.mean(np.square(np.array([y_average]*len(batch_y_regression)) - batch_y_regression))

        print ("val y gaussian field loss:", y_val_loss_fetched)

    step += 1
print "Optimization Finished!"